<a href="https://colab.research.google.com/github/victorog17/soulcode_aulas_pandas/blob/main/Soulcode_Pandas_006_pandera_tratamentodedados.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**BIBLIOTECAS**

In [ ]:
pip install pandera

In [ ]:
import pandas as pd
import pandera as pa

**EXTRAÇÃO DOS DADOS DE UMA/VARIAS FONTE(S)**

In [ ]:
df = pd.read_csv('/content/drive/MyDrive/Soul_Code_Academy/repositorio_pandas/ocorrencia_2010_2020 - ocorrencia_2010_2020.csv', sep=';', parse_dates=['ocorrencia_dia'], dayfirst=True)
df.head()

,codigo_ocorrencia,codigo_ocorrencia2,ocorrencia_classificacao,ocorrencia_cidade,ocorrencia_uf,ocorrencia_aerodromo,ocorrencia_dia,ocorrencia_hora,total_recomendacoes
0,40211,40211,INCIDENTE,RIO DE JANEIRO,RJ,****,2010-01-03,12:00:00,0
1,40349,40349,INCIDENTE,BELÉM,PA,SBBE,2010-01-03,11:05:00,0
2,40351,40351,INCIDENTE,RIO DE JANEIRO,RJ,SBRJ,2010-01-03,03:00:00,0
3,39527,39527,ACIDENTE,LUCAS DO RIO VERDE,MT,****,2010-01-04,17:30:00,0
4,40324,40324,INCIDENTE,PELOTAS,RS,SBPK,2010-01-05,19:25:00,0


In [ ]:
df.dtypes

codigo_ocorrencia                    int64
codigo_ocorrencia2                   int64
ocorrencia_classificacao            object
ocorrencia_cidade                   object
ocorrencia_uf                       object
ocorrencia_aerodromo                object
ocorrencia_dia              datetime64[ns]
ocorrencia_hora                     object
total_recomendacoes                  int64
dtype: object

In [ ]:
pd.unique(df['ocorrencia_uf'])

array(['RJ', 'PA', 'MT', 'RS', 'BA', 'AM', 'PR', 'MG', 'SP', 'DF', 'AC',
       'TO', 'SC', 'GO', 'MA', 'ES', 'PE', 'RR', 'MS', 'CE', 'AP', 'AL',
       'RO', 'SE', 'PB', 'PI', 'RN', '**'], dtype=object)

In [ ]:
df.loc[df['ocorrencia_uf'] == '**']

,codigo_ocorrencia,codigo_ocorrencia2,ocorrencia_classificacao,ocorrencia_cidade,ocorrencia_uf,ocorrencia_aerodromo,ocorrencia_dia,ocorrencia_hora,total_recomendacoes
2227,49474,49474,ACIDENTE,ÁGUAS INTERNACIONAIS,**,****,2013-09-02,02:54:00,0


In [ ]:
dfback = df.copy()

In [ ]:
dfback.head()

,codigo_ocorrencia,codigo_ocorrencia2,ocorrencia_classificacao,ocorrencia_cidade,ocorrencia_uf,ocorrencia_aerodromo,ocorrencia_dia,ocorrencia_hora,total_recomendacoes
0,40211,40211,INCIDENTE,RIO DE JANEIRO,RJ,****,2010-01-03,12:00:00,0
1,40349,40349,INCIDENTE,BELÉM,PA,SBBE,2010-01-03,11:05:00,0
2,40351,40351,INCIDENTE,RIO DE JANEIRO,RJ,SBRJ,2010-01-03,03:00:00,0
3,39527,39527,ACIDENTE,LUCAS DO RIO VERDE,MT,****,2010-01-04,17:30:00,0
4,40324,40324,INCIDENTE,PELOTAS,RS,SBPK,2010-01-05,19:25:00,0


Incosistencias:

'**', 

'****', 

'####', 

'###!', 

'*****'

A considerar: aerodromos '5JTI', '9PAA', '9PFC', '9PFX', '9PHD', '9PSD',

In [ ]:
df.loc[df['ocorrencia_aerodromo'] == '9PAA']

,codigo_ocorrencia,codigo_ocorrencia2,ocorrencia_classificacao,ocorrencia_cidade,ocorrencia_uf,ocorrencia_aerodromo,ocorrencia_dia,ocorrencia_hora,total_recomendacoes
3708,65543,65543,INCIDENTE,MACAÉ,RJ,9PAA,2016-08-05,18:00:00,0
5470,79380,79380,INCIDENTE,RIO DE JANEIRO,RJ,9PAA,2020-06-09,14:35:00,0
5710,79714,79714,INCIDENTE,RIO DE JANEIRO,RJ,9PAA,2020-12-08,14:50:00,0


**LIMPEZA/TRATAMENTO DOS DADOS**

In [ ]:
df.replace(['**', '****', '*****', '####', '###!'], pd.NA, inplace=True)

In [ ]:
df.isna().sum()

codigo_ocorrencia              0
codigo_ocorrencia2             0
ocorrencia_classificacao       0
ocorrencia_cidade              0
ocorrencia_uf                  1
ocorrencia_aerodromo        2349
ocorrencia_dia                 0
ocorrencia_hora                1
total_recomendacoes            0
dtype: int64

In [ ]:
df.drop_duplicates()

In [ ]:
df.dtypes

codigo_ocorrencia                    int64
codigo_ocorrencia2                   int64
ocorrencia_classificacao            object
ocorrencia_cidade                   object
ocorrencia_uf                       object
ocorrencia_aerodromo                object
ocorrencia_dia              datetime64[ns]
ocorrencia_hora                     object
total_recomendacoes                  int64
dtype: object

**VALIDAÇÃO DE DADOS**

In [ ]:
schema = pa.DataFrameSchema(
    columns = {
        "codigo_ocorrencia":pa.Column(pa.Int),
        "codigo_ocorrencia2":pa.Column(pa.Int),
        "ocorrencia_classificacao":pa.Column(pa.String),
        "ocorrencia_cidade":pa.Column(pa.String),
        "ocorrencia_uf":pa.Column(pa.String, pa.Check.str_length(2,2), nullable=True),
        "ocorrencia_aerodromo":pa.Column(pa.String, pa.Check.str_length(4,4), nullable=True),
        "ocorrencia_dia":pa.Column(pa.DateTime),
        "ocorrencia_hora":pa.Column(pa.String, pa.Check.str_matches(r'^([0-1]?[0-9]|[2][0-3]):([0-5][0-9]):([0-5][0-9])?$'), nullable=True),
        "total_recomendacoes":pa.Column(pa.Int)
    }
)

In [ ]:
schema.validate(df)

**FILTROS**

In [ ]:
df.isnull().sum()

codigo_ocorrencia              0
codigo_ocorrencia2             0
ocorrencia_classificacao       0
ocorrencia_cidade              0
ocorrencia_uf                  1
ocorrencia_aerodromo        2349
ocorrencia_dia                 0
ocorrencia_hora                1
total_recomendacoes            0
dtype: int64

In [ ]:
filter = df['ocorrencia_uf'].isnull()

In [ ]:
df.loc[filter]

,codigo_ocorrencia,codigo_ocorrencia2,ocorrencia_classificacao,ocorrencia_cidade,ocorrencia_uf,ocorrencia_aerodromo,ocorrencia_dia,ocorrencia_hora,total_recomendacoes
2227,49474,49474,ACIDENTE,ÁGUAS INTERNACIONAIS,<NA>,<NA>,2013-09-02,02:54:00,0


In [ ]:
df.count()

codigo_ocorrencia           5752
codigo_ocorrencia2          5752
ocorrencia_classificacao    5752
ocorrencia_cidade           5752
ocorrencia_uf               5751
ocorrencia_aerodromo        3403
ocorrencia_dia              5752
ocorrencia_hora             5751
total_recomendacoes         5752
dtype: int64

In [ ]:
filter1 = df['total_recomendacoes'] > 11
df.loc[filter1]

In [ ]:
df.loc[filter1, ['ocorrencia_cidade', 'ocorrencia_dia', 'total_recomendacoes']]

In [ ]:
filter2 = df['ocorrencia_classificacao'] == 'ACIDENTE'
filter3 = df['ocorrencia_uf'] == 'RJ'

In [ ]:
df.loc[filter2 & filter3]

In [ ]:
df.loc[filter2 | filter3]

In [ ]:
filter4 = df['ocorrencia_classificacao'].isin(['ACIDENTE', 'INCIDENTE'])
filter5 = df['ocorrencia_uf'] == 'SP'

In [ ]:
df.loc[filter4 & filter5]

In [ ]:
#filtroLetra = df['ocorrencia_cidade'].str[0] == 'C'
filtroLetra = df['ocorrencia_cidade'].str[-2:] == 'RA'
df.loc[filtroLetra]

In [ ]:
#filtroContem = df['ocorrencia_cidade'].str.contains('RIO | MA')
filtroContem = df['ocorrencia_cidade'].str.contains('RIO')
df.loc[filtroContem]

In [ ]:
filtroData = df['ocorrencia_dia'].dt.year == 2018
df.loc[filtroData & filter5 & filter2].shape

(27, 9)